In [ ]:
import tensorflow as tf
import numpy as np
import time
import copy

import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append("/home/shuo/Shuo/tensorflow_models/research")
sys.path.append("/home/shuo/Shuo/tensorflow_models/research/object_detection")
from tensorflow.core.framework import graph_pb2
from utils import label_map_util

from utils import visualization_utils as vis_util


In [ ]:
MODEL_ZOO = [
            'faster_rcnn_inception_resnet_v2_atrous_coco',                   # 0  .37
            'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco',      # 1  
            'faster_rcnn_inception_v2_coco',                                 # 2  .28
            'faster_rcnn_nas_coco',                                          # 3  .43
            'faster_rcnn_nas_lowproposals_coco',                             # 4  
            'faster_rcnn_resnet50_coco',                                     # 5  .30
            'faster_rcnn_resnet50_lowproposals_coco',                        # 6  
            'faster_rcnn_resnet101_coco',                                    # 7  .32
            'faster_rcnn_resnet101_kitti',                                   # 8  .87
            'faster_rcnn_resnet101_lowproposals_coco',                       # 9  
            'rfcn_resnet101_coco',                                           # 10 .30
            'ssd_inception_v2_coco',                                         # 11 .24
            'ssd_mobilenet_v1_coco'                                          # 12 .21
            ]  

USE_MODEL_ID = 12

MODEL_NAME = MODEL_ZOO[USE_MODEL_ID]

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/home/shuo/Shuo/tensorflow_models/research/object_detection/off_the_shelf_models/' + MODEL_NAME + '/frozen_inference_graph.pb'

# ssd_mobilenet
cut_node_1 = "Postprocessor/convert_scores"
cut_node_2 = "Postprocessor/ExpandDims_1"

# # faster_rcnn_resnet101_coco
# cut_node_1 = "SecondStageBoxPredictor/AvgPool"
# cut_node_2 = "SecondStagePostprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Slice_14"


def _node_name(n):
  if n.startswith("^"):
    return n[1:]
  else:
    return n.split(":")[0]


input_graph = tf.Graph()
with tf.Session(graph=input_graph):
    score = tf.placeholder(tf.float32, shape=(None, 1917, 90), name=cut_node_1)
    expand = tf.placeholder(tf.float32, shape=(None, 1917, 1, 4), name=cut_node_2)
    for node in input_graph.as_graph_def().node:
        if node.name == cut_node_1:
            score_def = node
        if node.name == cut_node_2:
            expand_def = node




In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    
    dest_nodes = [cut_node_1,cut_node_2]

    edges = {}
    name_to_node_map = {}
    node_seq = {}
    seq = 0
    for node in od_graph_def.node:
      n = _node_name(node.name)
#       print(n)
      name_to_node_map[n] = node
      edges[n] = [_node_name(x) for x in node.input]
      node_seq[n] = seq
      seq += 1
    print(len(od_graph_def.node))

    for d in dest_nodes:
      assert d in name_to_node_map, "%s is not in graph" % d

    nodes_to_keep = set()
    next_to_visit = dest_nodes[:]
    while next_to_visit:
      n = next_to_visit[0]
      del next_to_visit[0]
      if n in nodes_to_keep:
        continue
      nodes_to_keep.add(n)
      next_to_visit += edges[n]

    nodes_to_keep_list = sorted(list(nodes_to_keep), key=lambda n: node_seq[n])

    nodes_to_remove = set()
    for n in node_seq:
      if n in nodes_to_keep_list: continue
      nodes_to_remove.add(n)
    nodes_to_remove_list = sorted(list(nodes_to_remove), key=lambda n: node_seq[n])

    keep = graph_pb2.GraphDef()
    for n in nodes_to_keep_list:
      keep.node.extend([copy.deepcopy(name_to_node_map[n])])

    remove = graph_pb2.GraphDef()
    remove.node.extend([score_def])
    remove.node.extend([expand_def])
    for n in nodes_to_remove_list:
      remove.node.extend([copy.deepcopy(name_to_node_map[n])])

    with tf.device('/gpu:1'):
      tf.import_graph_def(keep, name='')
    with tf.device('/cpu:0'):
      tf.import_graph_def(remove, name='')
    print(len(keep.node))
    print(len(remove.node))
    for node in keep.node:
        print(node.name)
# NUM_CLASSES = 90
# label_map = label_map_util.load_labelmap('data/mscoco_label_map.pbtxt')
# categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
# category_index = label_map_util.create_category_index(categories)
PATH_TO_LABELS = os.path.join('/home/shuo/Shuo/tensorflow_models/research/object_detection/data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
if MODEL_NAME[-5:] == 'kitti':
    PATH_TO_LABELS = os.path.join('/home/shuo/Shuo/tensorflow_models/research/object_detection/data', 'kitti_label_map.pbtxt')
    NUM_CLASSES = 2
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
    
    
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/shuo/Shuo/tensorflow_models/research/object_detection/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
from tensorflow.python.client import timeline

with detection_graph.as_default():
  with tf.Session(graph=detection_graph,config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    score_out = detection_graph.get_tensor_by_name('Postprocessor/convert_scores:0')
    expand_out = detection_graph.get_tensor_by_name('Postprocessor/ExpandDims_1:0')
    score_in = detection_graph.get_tensor_by_name('Postprocessor/convert_scores_1:0')
    expand_in = detection_graph.get_tensor_by_name('Postprocessor/ExpandDims_1_1:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    i = 0
#     for image_path in TEST_IMAGE_PATHS:
    for _ in range(10):
      image_path = TEST_IMAGE_PATHS[1]
      i += 1
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      image_np_expanded = np.expand_dims(image_np, axis=0)
#       options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#       run_metadata = tf.RunMetadata()
      start_time = time.time()
      
      (score, expand) = sess.run([score_out, expand_out], feed_dict={image_tensor: image_np_expanded})
#       (boxes, scores, classes, num) = sess.run(\
#             [detection_boxes, detection_scores, detection_classes, num_detections], \
#             feed_dict={score_in:score, expand_in: expand}, \
#             options=options, run_metadata=run_metadata)
      (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={score_in:score, expand_in: expand})
      print('Iteration %d: %.3f sec'%(i, time.time()-start_time))

      vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
       np.squeeze(boxes),
      np.squeeze(classes).astype(np.int32),
      np.squeeze(scores),
      category_index,
      use_normalized_coordinates=True,
      line_thickness=8)

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    
    
# #     Create the Timeline object, and write it to a json file
#     fetched_timeline = timeline.Timeline(run_metadata.step_stats)
#     chrome_trace = fetched_timeline.generate_chrome_trace_format()
#     with open('Experiment3.json' , 'w') as f:
#       f.write(chrome_trace)